In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cvxpy as cp
import numpy as np
import torch
import torch.nn as nn

from pideq.net import PIDEQ
from pideq.utils import load_from_wandb

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
deq = load_from_wandb(PIDEQ(np.pi / 2, n_states=200), '2qka16vc', project='pideq-nls', model_fname='model_best')
deq

PIDEQ(
  (A): Linear(in_features=200, out_features=200, bias=True)
  (B): Linear(in_features=2, out_features=200, bias=True)
  (C): Linear(in_features=200, out_features=2, bias=True)
  (D): Linear(in_features=2, out_features=2, bias=True)
)

In [12]:
A0 = deq.A.weight.detach().numpy()
np.linalg.norm(A0, ord=np.inf)

7.147842

### Full optimization

In [25]:
kappa = .9
A = cp.Variable(A0.shape, complex=False)

constraint = [cp.norm(A, p='inf') <= kappa,]

prob = cp.Problem(cp.Minimize(cp.norm(A - A0, p='fro')), constraint)
prob.solve(verbose=False)
prob.status

'optimal'

In [26]:
np.linalg.norm(A.value - A0, ord='fro')

2.769859299411888

### Decomposition in rows

In [27]:
kappa = .9
A_rows = list()

for i in range(A0.shape[0]):
    a = cp.Variable(A0.shape[1], complex=False)

    constraint = [cp.norm(a, p=1) <= kappa,]

    prob = cp.Problem(cp.Minimize(cp.norm(a - A0[i], p=2)), constraint)
    prob.solve(verbose=False)

    A_rows.append(a.value)

In [28]:
A = np.vstack(A_rows)
np.linalg.norm(A, ord=np.inf)

0.8999999999421489

In [29]:
np.linalg.norm(A - A0, ord='fro')

2.769859300913568